In [ ]:
!pip install Flask-SQLAlchemy

from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

db = SQLAlchemy() # You'd initialize this with your Flask app

class Claim(db.Model):
    __tablename__ = 'claims'
    claim_id = db.Column(db.Integer, primary_key=True)
    claim_text = db.Column(db.String(1000), nullable=False)
    date_of_claim = db.Column(db.DateTime)
    rating = db.Column(db.String(100)) # E.g., "False", "Misleading"
    debunking_details = db.Column(db.Text, nullable=False) # Can store HTML/Markdown
    # For simplicity, storing links as comma-separated strings. A related table is better for complex cases.
    evidence_links = db.Column(db.Text) # URLs separated by commas or newlines
    reason_for_spread = db.Column(db.Text)
    publication_date = db.Column(db.DateTime, default=datetime.utcnow)
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    geopolitical_context = db.Column(db.Text)
    # Relationship (many-to-many with Source, if a claim can involve multiple sources directly)
    # This requires an association table, which we can detail if you'd like.
    # For now, let's keep it simpler and imagine you'd manually link or mention sources in debunking_details.

    def __repr__(self):
        return f'<Claim {self.claim_text[:50]}...>'

class Source(db.Model):
    __tablename__ = 'sources'
    source_id = db.Column(db.Integer, primary_key=True)
    source_name = db.Column(db.String(255), nullable=False, unique=True)
    source_url = db.Column(db.String(500))
    source_type = db.Column(db.String(100)) # E.g., "Mainstream Media", "Blog"
    profile_details = db.Column(db.Text) # Can store HTML/Markdown
    # For documented falsehoods, you could have a text field or link to Claim entries.
    documented_falsehoods_summary = db.Column(db.Text)
    biases = db.Column(db.Text)
    reliability_score = db.Column(db.String(50)) # E.g., "Low", "Medium", "High", or a numeric score
    reliability_summary = db.Column(db.Text)
    criteria_for_score = db.Column(db.Text) # Your transparent criteria explanation
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    def __repr__(self):
        return f'<Source {self.source_name}>'

# --- Helper for initializing the database (run once) ---
# def init_db(app):
#     with app.app_context():
#         db.create_all()

In [ ]:
from flask import Blueprint, request, redirect, url_for, render_template #, flash
# Assuming you have your db and models imported (from .models import db, Claim, Source)

admin_bp = Blueprint('admin', __name__, url_prefix='/admin')

# --- Claim Management ---
@admin_bp.route('/claim/new', methods=['GET', 'POST'])
def new_claim():
    if request.method == 'POST':
        # Get data from form
        claim_text = request.form.get('claim_text')
        # ... (get all other fields: date_of_claim, rating, etc.)
        debunking_details = request.form.get('debunking_details')
        evidence_links = request.form.get('evidence_links') # Could be a list
        reason_for_spread = request.form.get('reason_for_spread')
        geopolitical_context = request.form.get('geopolitical_context')
        rating = request.form.get('rating')

        # Basic validation (you'd add more)
        if not claim_text or not debunking_details:
            # flash('Claim text and debunking details are required.', 'error')
            return render_template('admin/edit_claim.html', form_data=request.form) # Pass back form data

        new_claim_obj = Claim(
            claim_text=claim_text,
            # date_of_claim= # parse date from form
            rating=rating,
            debunking_details=debunking_details,
            evidence_links=evidence_links,
            reason_for_spread=reason_for_spread,
            geopolitical_context=geopolitical_context
        )
        db.session.add(new_claim_obj)
        db.session.commit()
        # flash('Claim created successfully!', 'success')
        return redirect(url_for('admin.list_claims')) # Redirect to a list of claims
    return render_template('admin/edit_claim.html') # Show an empty form

@admin_bp.route('/claims')
def list_claims():
    claims = Claim.query.order_by(Claim.publication_date.desc()).all()
    return render_template('admin/list_claims.html', claims=claims)

# You would also need routes for:
# - Editing an existing claim: /claim/edit/<claim_id>
# - Deleting a claim: /claim/delete/<claim_id> (usually via POST)

# --- Source Spotlight Management (Crucial Feature Focus) ---
@admin_bp.route('/source/new', methods=['GET', 'POST'])
def new_source():
    if request.method == 'POST':
        source_name = request.form.get('source_name')
        source_url = request.form.get('source_url')
        source_type = request.form.get('source_type')
        profile_details = request.form.get('profile_details')
        documented_falsehoods_summary = request.form.get('documented_falsehoods_summary')
        biases = request.form.get('biases')
        reliability_score = request.form.get('reliability_score')
        reliability_summary = request.form.get('reliability_summary')
        criteria_for_score = request.form.get('criteria_for_score')

        if not source_name or not profile_details:
            # flash('Source name and profile details are required.', 'error')
            return render_template('admin/edit_source.html', form_data=request.form)

        new_source_obj = Source(
            source_name=source_name,
            source_url=source_url,
            source_type=source_type,
            profile_details=profile_details,
            documented_falsehoods_summary=documented_falsehoods_summary,
            biases=biases,
            reliability_score=reliability_score,
            reliability_summary=reliability_summary,
            criteria_for_score=criteria_for_score
        )
        db.session.add(new_source_obj)
        db.session.commit()
        # flash('Source profile created successfully!', 'success')
        return redirect(url_for('admin.list_sources'))
    return render_template('admin/edit_source.html') # Show an empty form for a new source

@admin_bp.route('/sources')
def list_sources():
    sources = Source.query.order_by(Source.source_name.asc()).all()
    return render_template('admin/list_sources.html', sources=sources)

# You would also need routes for:
# - Editing an existing source: /source/edit/<source_id>
# - Deleting a source: /source/delete/<source_id> (usually via POST)

In [ ]:
# ipython-input-7-21f1ef51d8b4 - models.py part
!pip install Flask-SQLAlchemy

from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from flask import Flask, Blueprint, request, redirect, url_for, render_template #, flash

# Initialize db here, but it needs the app context later
db = SQLAlchemy()

class Claim(db.Model):
    __tablename__ = 'claims'
    claim_id = db.Column(db.Integer, primary_key=True)
    claim_text = db.Column(db.String(1000), nullable=False)
    date_of_claim = db.Column(db.DateTime)
    rating = db.Column(db.String(100)) # E.g., "False", "Misleading"
    debunking_details = db.Column(db.Text, nullable=False) # Can store HTML/Markdown
    # For simplicity, storing links as comma-separated strings. A related table is better for complex cases.
    evidence_links = db.Column(db.Text) # URLs separated by commas or newlines
    reason_for_spread = db.Column(db.Text)
    publication_date = db.Column(db.DateTime, default=datetime.utcnow)
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    geopolitical_context = db.Column(db.Text)

    def __repr__(self):
        return f'<Claim {self.claim_text[:50]}...>'

class Source(db.Model):
    __tablename__ = 'sources'
    source_id = db.Column(db.Integer, primary_key=True)
    source_name = db.Column(db.String(255), nullable=False, unique=True)
    source_url = db.Column(db.String(500))
    source_type = db.Column(db.String(100)) # E.g., "Mainstream Media", "Blog"
    profile_details = db.Column(db.Text) # Can store HTML/Markdown
    documented_falsehoods_summary = db.Column(db.Text)
    biases = db.Column(db.Text)
    reliability_score = db.Column(db.String(50)) # E.g., "Low", "Medium", "High", or a numeric score
    reliability_summary = db.Column(db.Text)
    criteria_for_score = db.Column(db.Text) # Your transparent criteria explanation
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    def __repr__(self):
        return f'<Source {self.source_name}>'

# ipython-input-3-21f1ef51d8b4 - routes_admin.py part
# Now that Claim and Source are defined in the same cell, we don't need the commented import
# from .models import db, Claim, Source

admin_bp = Blueprint('admin', __name__, url_prefix='/admin')

# --- Claim Management ---
@admin_bp.route('/claim/new', methods=['GET', 'POST'])
def new_claim():
    if request.method == 'POST':
        # Get data from form
        claim_text = request.form.get('claim_text')
        # ... (get all other fields: date_of_claim, rating, etc.)
        debunking_details = request.form.get('debunking_details')
        evidence_links = request.form.get('evidence_links') # Could be a list
        reason_for_spread = request.form.get('reason_for_spread')
        geopolitical_context = request.form.get('geopolitical_context')
        rating = request.form.get('rating')

        # Basic validation (you'd add more)
        if not claim_text or not debunking_details:
            # flash('Claim text and debunking details are required.', 'error')
            return render_template('admin/edit_claim.html', form_data=request.form) # Pass back form data

        new_claim_obj = Claim(
            claim_text=claim_text,
            # date_of_claim= # parse date from form
            rating=rating,
            debunking_details=debunking_details,
            evidence_links=evidence_links,
            reason_for_spread=reason_for_spread,
            geopolitical_context=geopolitical_context
        )
        db.session.add(new_claim_obj)
        db.session.commit()
        # flash('Claim created successfully!', 'success')
        return redirect(url_for('admin.list_claims')) # Redirect to a list of claims
    return render_template('admin/edit_claim.html') # Show an empty form

@admin_bp.route('/claims')
def list_claims():
    claims = Claim.query.order_by(Claim.publication_date.desc()).all()
    return render_template('admin/list_claims.html', claims=claims)

# ipython-input-7-21f1ef51d8b4 - models.py part
!pip install Flask-SQLAlchemy

from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from flask import Flask, Blueprint, request, redirect, url_for, render_template #, flash

# Initialize db here, but it needs the app context later
db = SQLAlchemy()

class Claim(db.Model):
    __tablename__ = 'claims'
    claim_id = db.Column(db.Integer, primary_key=True)
    claim_text = db.Column(db.String(1000), nullable=False)
    date_of_claim = db.Column(db.DateTime)
    rating = db.Column(db.String(100)) # E.g., "False", "Misleading"
    debunking_details = db.Column(db.Text, nullable=False) # Can store HTML/Markdown
    # For simplicity, storing links as comma-separated strings. A related table is better for complex cases.
    evidence_links = db.Column(db.Text) # URLs separated by commas or newlines
    reason_for_spread = db.Column(db.Text)
    publication_date = db.Column(db.DateTime, default=datetime.utcnow)
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    geopolitical_context = db.Column(db.Text)

    def __repr__(self):
        return f'<Claim {self.claim_text[:50]}...>'

class Source(db.Model):
    __tablename__ = 'sources'
    source_id = db.Column(db.Integer, primary_key=True)
    source_name = db.Column(db.String(255), nullable=False, unique=True)
    source_url = db.Column(db.String(500))
    source_type = db.Column(db.String(100)) # E.g., "Mainstream Media", "Blog"
    profile_details = db.Column(db.Text) # Can store HTML/Markdown
    documented_falsehoods_summary = db.Column(db.Text)
    biases = db.Column(db.Text)
    reliability_score = db.Column(db.String(50)) # E.g., "Low", "Medium", "High", or a numeric score
    reliability_summary = db.Column(db.Text)
    criteria_for_score = db.Column(db.Text) # Your transparent criteria explanation
    last_updated_date = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    def __repr__(self):
        return f'<Source {self.source_name}>'

# ipython-input-3-21f1ef51d8b4 - routes_admin.py part
# Now that Claim and Source are defined in the same cell, we don't need the commented import
# from .models import db, Claim, Source

admin_bp = Blueprint('admin', __name__, url_prefix='/admin')

# --- Claim Management ---
@admin_bp.route('/claim/new', methods=['GET', 'POST'])
def new_claim():
    if request.method == 'POST':
        # Get data from form
        claim_text = request.form.get('claim_text')
        # ... (get all other fields: date_of_claim, rating, etc.)
        debunking_details = request.form.get('debunking_details')
        evidence_links = request.form.get('evidence_links') # Could be a list
        reason_for_spread = request.form.get('reason_for_spread')
        geopolitical_context = request.form.get('geopolitical_context')
        rating = request.form.get('rating')

        # Basic validation (you'd add more)
        if not claim_text or not debunking_details:
            # flash('Claim text and debunking details are required.', 'error')
            return render_template('admin/edit_claim.html', form_data=request.form) # Pass back form data

        new_claim_obj = Claim(
            claim_text=claim_text,
            # date_of_claim= # parse date from form
            rating=rating,
            debunking_details=debunking_details,
            evidence_links=evidence_links,
            reason_for_spread=reason_for_spread,
            geopolitical_context=geopolitical_context
        )
        db.session.add(new_claim_obj)
        db.session.commit()
        # flash('Claim created successfully!', 'success')
        return redirect(url_for('admin.list_claims')) # Redirect to a list of claims
    return render_template('admin/edit_claim.html') # Show an empty form

@admin_bp.route('/claims')
def list_claims():
    claims = Claim.query.order_by(Claim.publication_date.desc()).all()
    return render_template('admin/list_claims.html', claims=claims)


# --- Source Spotlight Management (Crucial Feature Focus) ---
@admin_bp.route('/source/new', methods=['GET', 'POST'])
def new_source():
    pass

<div class="md-recitation">
  Sources
  <ol>
  <li><a href="https://github.com/bmliddicoat/Inventory_Management_Concept">https://github.com/bmliddicoat/Inventory_Management_Concept</a></li>
  </ol>
</div>